### 1 - Importing necessary libraries and data

In [20]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import joblib
import pandas as pd

In [19]:
df = pd.read_csv("../data/raw/prompts_v1.csv")

NameError: name 'pd' is not defined

### 2 - Loading the models

In [2]:
import joblib

cluster_LR = joblib.load("../models/ML Models/cluster_LR_model.pkl")
cluster_encoder = joblib.load("../models/ML Models/cluster_label_encoder.pkl")

subclass_LR = joblib.load("../models/ML Models/subclass_LR_model.pkl")
subclass_encoder = joblib.load("../models/ML Models/subclass_label_encoder.pkl")

vectorizer = joblib.load("../models/ML Models/tfidf_vectorizer.pkl")

### 3 - Function to predict clusters and prompts

In [21]:
def get_cluster_from_subclass(subclass):
    cluster = df[df["sub_class"] == subclass]["cluster"].values[0]
    return cluster

In [22]:
def classify_prompt(new_prompt):

    X_new = vectorizer.transform([new_prompt])

    cluster_probabilities = cluster_LR.predict_proba(X_new)
    subclass_probabilities = subclass_LR.predict_proba(X_new)

    cluster_predicted_index = cluster_probabilities.argmax(axis=1)
    subclass_predicted_index = subclass_probabilities.argmax(axis=1)

    cluster_confidence_score = cluster_probabilities[0][cluster_predicted_index]

    subclass_confidence_score = subclass_probabilities[0][subclass_predicted_index]
    predicted_subclass_label = subclass_encoder.inverse_transform(
        subclass_predicted_index
    )

    if subclass_confidence_score > 0.55:
        predicted_cluster_label = get_cluster_from_subclass(predicted_cluster_label[0])
        return f"Subclass: {predicted_cluster_label[0]} | Cluster: {predicted_cluster_label}"

    else:
        predicted_cluster_label = cluster_encoder.inverse_transform(
            cluster_predicted_index
        )
        if cluster_confidence_score > 0.45:
            return f"Cluster: {predicted_cluster_label[0]} | Subclass: N/A (Suggested: {predicted_subclass_label})"

        return "General Model"

In [24]:
classify_prompt("How to use OpenAI's API within Streamlit?")

"Cluster: Programming and Development | Subclass: N/A (Suggested: ['API Integration'])"

In [28]:
classify_prompt("How to write an email to my team lead?")

"Cluster: Business and Productivity | Subclass: N/A (Suggested: ['Email Generation'])"

In [29]:
classify_prompt("Help a friend with depression")

'General Model'